# Imports

In [2]:
# %pip install ultralytics

In [1]:
from nb_utils import set_root
PROJECT_DIR = set_root(2)

In [2]:
import numpy as np
import pandas as pd
import supervision as sv

import ultralytics
from ultralytics import YOLO
ultralytics.checks()

Ultralytics YOLOv8.2.103 🚀 Python-3.10.12 torch-2.4.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Ti, 8188MiB)
Setup complete ✅ (16 CPUs, 15.6 GB RAM, 111.7/1006.9 GB disk)


# Parameters

In [3]:
path_data = PROJECT_DIR / 'data'
path_intermediate = path_data / "02_intermediate"

file_path_tracker = path_intermediate / 'tracker.csv'

# Treinamento do Modelo

In [19]:
# # Load a model
# model = YOLO('yolov8s.yaml')  # build a new model from scratch
# model = YOLO('yolov8s.pt')  # load a pretrained model (recommended for training)

# # Use the model
# results = model.train(data='data.yaml', epochs=3)  # train the model
# results = model.val()  # evaluate model performance on the validation set

**Detecção em Vídeo**

In [ ]:

# Inicializando o modelo, o rastreador e os anotadores
model = YOLO('runs/detect/train/weights/best.pt')
tracker = sv.ByteTrack()
annotator = sv.RoundBoxAnnotator()
label_annotator = sv.LabelAnnotator()
trace_annotator = sv.TraceAnnotator()

# Criando um DataFrame vazio para armazenar as detecções
df_detections = pd.DataFrame(columns=["tracker_id", "class_id", "x_min", "y_min", "x_max", "y_max"])

# Função de callback para processar cada frame
def callback(frame: np.ndarray, _: int) -> np.ndarray:
    global df_detections

    # Fazendo a predição no frame atual
    results = model(frame)[0]

    # Convertendo resultados para o formato de detecção
    detections = sv.Detections.from_ultralytics(results)

    # Atualizando os rastreamentos com o ByteTrack
    detections = tracker.update_with_detections(detections)
    
    # Extraindo informações das detecções para o DataFrame
    new_data = []
    for i, (class_id, tracker_id, box) in enumerate(zip(detections.class_id, detections.tracker_id, detections.xyxy)):
        # Extraindo coordenadas da bounding box
        x_min, y_min, x_max, y_max = box
        
        # Adicionando as informações à lista de novas linhas
        new_data.append({
            "tracker_id": tracker_id,
            "class_id": class_id,
            "x_min": x_min,
            "y_min": y_min,
            "x_max": x_max,
            "y_max": y_max
        })

    # Concatenando novas linhas ao DataFrame
    df_detections = pd.concat([df_detections, pd.DataFrame(new_data)], ignore_index=True)

    # Criando rótulos para cada detecção
    labels = [
        f"#{tracker_id} {results.names[class_id]}"
        for class_id, tracker_id
        in zip(detections.class_id, detections.tracker_id)
    ]

    # Anotando o frame com as caixas de detecção, rótulos e rastros
    annotated_frame = annotator.annotate(frame.copy(), detections=detections)
    annotated_frame = label_annotator.annotate(annotated_frame, detections=detections, labels=labels)
    return trace_annotator.annotate(annotated_frame, detections=detections)

# Processando o vídeo e salvando o resultado
sv.process_video(
    source_path="11.2.mp4",
    target_path="test.mp4",#"resultIDTraceRo.mp4",
    callback=callback
)

# Exibindo as primeiras linhas do DataFrame ao final do processamento
print(df_detections.head())


In [8]:
import pandas as pd
import numpy as np
import supervision as sv
from ultralytics import YOLO


class VideoProcessor:
    def __init__(self, model_path: str, source_path: str, target_path: str):
        self.model = YOLO(model_path)
        self.tracker = sv.ByteTrack()
        self.annotator = sv.RoundBoxAnnotator()
        self.label_annotator = sv.LabelAnnotator()
        self.trace_annotator = sv.TraceAnnotator()
        self.df_detections = pd.DataFrame(columns=["tracker_id", "class_id", "x_min", "y_min", "x_max", "y_max"])
        self.source_path = source_path
        self.target_path = target_path

    def callback(self, frame: np.ndarray, _: int) -> np.ndarray:
        results = self.model(frame)[0]
        detections = sv.Detections.from_ultralytics(results)
        detections = self.tracker.update_with_detections(detections)
        
        new_data = []
        for class_id, tracker_id, box in zip(detections.class_id, detections.tracker_id, detections.xyxy):
            x_min, y_min, x_max, y_max = box
            new_data.append({
                "tracker_id": tracker_id,
                "class_id": class_id,
                "x_min": x_min,
                "y_min": y_min,
                "x_max": x_max,
                "y_max": y_max
            })

        self.df_detections = pd.concat([self.df_detections, pd.DataFrame(new_data)], ignore_index=True)

        labels = [
            f"#{tracker_id} {results.names[class_id]}"
            for class_id, tracker_id in zip(detections.class_id, detections.tracker_id)
        ]

        annotated_frame = self.annotator.annotate(frame.copy(), detections=detections)
        annotated_frame = self.label_annotator.annotate(annotated_frame, detections=detections, labels=labels)
        return self.trace_annotator.annotate(annotated_frame, detections=detections)

    def process_video(self):
        sv.process_video(
            source_path=self.source_path,
            target_path=self.target_path,
            callback=self.callback
        )

# Usage
processor = VideoProcessor(
    model_path='runs/detect/train/weights/best.pt',
    source_path='11.2.mp4',
    target_path='test.mp4'
)
processor.process_video()


0: 416x640 24 sperms, 7.3ms
Speed: 1.1ms preprocess, 7.3ms inference, 1.2ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 25 sperms, 6.9ms
Speed: 1.7ms preprocess, 6.9ms inference, 1.2ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 22 sperms, 6.4ms
Speed: 1.2ms preprocess, 6.4ms inference, 1.1ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 24 sperms, 6.5ms
Speed: 1.2ms preprocess, 6.5ms inference, 1.3ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 26 sperms, 6.4ms
Speed: 1.3ms preprocess, 6.4ms inference, 1.2ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 28 sperms, 6.4ms
Speed: 1.3ms preprocess, 6.4ms inference, 1.2ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 28 sperms, 6.4ms
Speed: 1.3ms preprocess, 6.4ms inference, 1.2ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 26 sperms, 6.2ms
Speed: 1.2ms preprocess, 6.2ms inference, 1.1ms postprocess per image at shape (1, 3, 416, 640)


Speed: 1.7ms preprocess, 6.6ms inference, 1.2ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 27 sperms, 6.2ms
Speed: 1.4ms preprocess, 6.2ms inference, 2.1ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 27 sperms, 10.5ms
Speed: 1.3ms preprocess, 10.5ms inference, 2.6ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 23 sperms, 7.5ms
Speed: 1.5ms preprocess, 7.5ms inference, 1.3ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 20 sperms, 6.7ms
Speed: 1.3ms preprocess, 6.7ms inference, 1.2ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 19 sperms, 8.5ms
Speed: 1.4ms preprocess, 8.5ms inference, 1.5ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 28 sperms, 7.1ms
Speed: 1.4ms preprocess, 7.1ms inference, 1.9ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 27 sperms, 7.2ms
Speed: 1.5ms preprocess, 7.2ms inference, 1.3ms postprocess per image at shape (1, 3, 416, 640)

0: 416x640 27 sperms, 6.5m

In [9]:
processor.df_detections

,tracker_id,class_id,x_min,y_min,x_max,y_max
0,345,0,647.209595,589.079590,685.848877,632.157288
1,346,0,319.812164,370.250580,355.217896,410.914764
2,347,0,1267.440918,452.922394,1309.288208,494.093994
3,348,0,146.338654,707.259155,188.505234,752.012329
4,349,0,873.423035,373.475739,914.815857,412.739807
...,...,...,...,...,...,...
1821,580,0,150.290985,61.459763,193.075760,103.115036
1822,652,0,0.000000,615.464905,18.856745,652.565613
1823,357,0,0.000000,703.535461,16.553658,738.090881
1824,421,0,976.253113,505.177704,1008.852539,543.480530


# Save

In [ ]:
# df_detections#.to_csv(file_path_tracker, index=False)
processor.df_detections